In [1]:
print("ok")

ok


In [2]:
%pwd

'e:\\Code\\gen-ai\\Medical-chatbot\\Medical_ChatBot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'e:\\Code\\gen-ai\\Medical-chatbot\\Medical_ChatBot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
#Extract Data from pdf file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader
                             )
    documents = loader.load()
    
    return documents

In [15]:
extracted_data = load_pdf_file(data="Data/")

In [23]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [24]:
text_chunks = text_split(extracted_data)
print(f"Length of text chunks {len(text_chunks)}")

Length of text chunks 10484


In [25]:
from langchain.embeddings import HuggingFaceEmbeddings

In [26]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [27]:
embeddings = download_hugging_face_embeddings()

C:\Users\Asus\AppData\Local\Temp\ipykernel_31424\1561642224.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
e:\Code\gen-ai\Medical-chatbot\Medical_ChatBot\testenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Code\gen-ai\Medical-chatbot\Medical_ChatBot\testenv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_down

In [28]:
query_result = embeddings.embed_query("Hello World")
print(f"Length: {len(query_result)}")

Length: 384


In [76]:
from dotenv import load_dotenv
load_dotenv()

True

In [77]:
import os
os.environ['PINECONE_API_KEY'] = os.getenv("PINECONE_API_KEY")
os.environ['COHERE_API_KEY'] = os.environ.get('COHERE_API_KEY')

In [41]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone()
index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [42]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [43]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore(
    index_name=index_name,
    embedding=embeddings
)

In [79]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [80]:
retriever_docs = retriever.invoke("What is acne?")

In [81]:
retriever_docs

[Document(id='2db35970-0c30-42ad-b1e2-e21b17f29766', metadata={'page': 623.0, 'source': 'Data\\knowlegde_base.pdf'}, page_content='treatment of acne also may have a role in the treatment ofrosacea. Accumulating evidence suggests that topicalisotretinoin and topical azelaic acid can reduce the rednessand pimples. Some patients who use these medicationsexperience skin irritation that tends to resolve with time.\nFor later stages of the disorder, a surgical procedure'),
 Document(id='ec13b087-7f2a-4366-97cc-3f9fc0914dcb', metadata={'page': 622.0, 'source': 'Data\\knowlegde_base.pdf'}, page_content='and cysts, however, areseen in acne patients, but not in those with rosacea.'),
 Document(id='a658ab00-cbc3-46d0-9fc6-8a6bf9afd99c', metadata={'page': 624.0, 'source': 'Data\\knowlegde_base.pdf'}, page_content='Helm, Klaus F., and James G. Marks, Jr. Atlas of Differential\nDiagnosis in Dermatology. New York: Churchill Living-\nstone, 1998.\nMacsai, Marian S., et al. “Acne Rosacea.” In Eye and S

In [73]:
from langchain.llms import Cohere
llm = Cohere(max_tokens=500, temperature=0.4)

In [83]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question answering tasks, "
    "Use the following pieces of retrieved context to answer "
    "the question. If you dont know the answer say that you "
    "dont know the answer. Use three sentences maximum and keep the "
    "answer concise. "
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [84]:
question_answer_chain =  create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [90]:
response = rag_chain.invoke({"input": "What is IBS?"})
response['answer']

' IBS, or irritable bowel syndrome, is a condition that affects the large intestine. It is characterized by symptoms such as abdominal pain, bloating, and changes in bowel habits. IBS can be triggered by stress or certain foods and is generally treated with dietary changes and medications. '